In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import utils

In [ ]:
from textblob import TextBlob
import nltk

nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
text = open('/content/drive/My Drive/shakespeare.txt').read().splitlines()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
text

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
vocab_size = len(tokenizer.word_index)+1
vocab_size

3201

In [ ]:
input_sequence = []
for sent in text :
  sequence = tokenizer.texts_to_sequences([sent])[0]
  for j in range(1,len(sequence)) :
    n_gram = sequence[:j+1]
    input_sequence.append(n_gram)

In [ ]:
max_len = max([len(i) for i in input_sequence])
input_sequence = pad_sequences(input_sequence, maxlen=max_len)
input_sequence

array([[   0,    0,    0, ...,    0,    2, 1372],
       [   0,    0,    0, ...,    0,   31, 1373],
       [   0,    0,    0, ...,   31, 1373, 1374],
       ...,
       [   0,    0,    0, ...,  486, 3200,   15],
       [   0,    0,    0, ..., 3200,   15,   14],
       [   0,    0,    0, ...,    0,    2,  200]], dtype=int32)

In [ ]:
input = input_sequence[:,:-1]
labels = input_sequence[:,-1]
labels = utils.to_categorical(labels,num_classes=vocab_size)
labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size,100,input_length=max_len-1))
model.add(LSTM(128,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dense(vocab_size,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 100)           320100    
                                                                 
 lstm (LSTM)                 (None, 10, 128)           117248    
                                                                 
 dropout (Dropout)           (None, 10, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 3201)              412929    
                                                                 
Total params: 981861 (3.75 MB)
Trainable params: 981861 (3.75 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
history = model.fit(input,labels,epochs=200)

Epoch 1/200
485/485 [==============================] - 17s 21ms/step - loss: 6.8776 - accuracy: 0.0210
Epoch 2/200
485/485 [==============================] - 6s 11ms/step - loss: 6.4947 - accuracy: 0.0251
Epoch 3/200
485/485 [==============================] - 4s 8ms/step - loss: 6.4000 - accuracy: 0.0306
Epoch 4/200
485/485 [==============================] - 4s 8ms/step - loss: 6.3018 - accuracy: 0.0340
Epoch 5/200
485/485 [==============================] - 5s 10ms/step - loss: 6.1821 - accuracy: 0.0382
Epoch 6/200
485/485 [==============================] - 4s 7ms/step - loss: 6.0615 - accuracy: 0.0421
Epoch 7/200
485/485 [==============================] - 4s 8ms/step - loss: 5.9459 - accuracy: 0.0443
Epoch 8/200
485/485 [==============================] - 5s 9ms/step - loss: 5.8207 - accuracy: 0.0510
Epoch 9/200
485/485 [==============================] - 4s 8ms/step - loss: 5.6858 - accuracy: 0.0563
Epoch 10/200
485/485 [==============================] - 4s 8ms/step - loss: 5.5494 - ac

In [ ]:
import numpy as np

def sample(preds, temperature=1.0):
    """ Helper function to sample an index from a probability array. """
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-10) / temperature  # Adding a small number to avoid log(0)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

new_text = 'For having traffic with thy self alone'
words_per_line = 8  # Rough estimate of words per line in a sonnet
total_lines = 14  # Total lines in a sonnet
sonnet = []
current_line = []

for _ in range(words_per_line * total_lines):
    tokens = tokenizer.texts_to_sequences([new_text])
    pad = pad_sequences(tokens, maxlen=max_len-1)
    preds = model.predict(pad)[0]
    next_index = sample(preds, temperature=0.5)  # Using temperature to add randomness
    next_word = tokenizer.index_word.get(next_index, 'unknown')

    current_line.append(next_word)
    new_text += ' ' + next_word

    # Check if current line has enough words
    if len(current_line) >= words_per_line:
        sonnet.append(" ".join(current_line))
        current_line = []  # Reset current line

# Output the sonnet
for line in sonnet:
    print(line)

1/1 [==============================] - 0s 18ms/step
loved more rare ' night ' deeds thee
bright fear sheds else ' fear ' son
' brow grief brow cheeks bright lose one
' eyes ' erred great state ' free
cheeks fair verse prove bright 'will' still date
force say this to thine indeed free free
free great fear ' show mine decay staineth
spent spent bright eyes bright eyes thee have
stand sheds cheeks bright truth ' spent '
shown great fear ' view is old state
one ' sit pride life are all old
heir verse words heart too flatter some eyes
' lie are truth that erred due and
every dispatch hours die see tongue grew staineth


In [ ]:
import numpy as np

def sample(preds, temperature=1.0):
    """ Helper function to sample an index from a probability array. """
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-10) / temperature  # Adding a small number to avoid log(0)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

new_text = 'For your lovers sake'
words_per_line = 8  # Rough estimate of words per line in a sonnet
total_lines = 14  # Total lines in a sonnet
sonnet = []
current_line = []

for _ in range(words_per_line * total_lines):
    tokens = tokenizer.texts_to_sequences([new_text])
    pad = pad_sequences(tokens, maxlen=max_len-1)
    preds = model.predict(pad)[0]
    next_index = sample(preds, temperature=0.5)  # Using temperature to add randomness
    next_word = tokenizer.index_word.get(next_index, 'unknown')

    current_line.append(next_word)
    new_text += ' ' + next_word

    # Check if current line has enough words
    if len(current_line) >= words_per_line:
        sonnet.append(" ".join(current_line))
        current_line = []  # Reset current line

# Output the sonnet
for line in sonnet:
    print(line)

1/1 [==============================] - 0s 18ms/step
can not so are but new waste dead
must strive free free shown bright fear each
brow praise pride ' this verse sad sky
soul live new stand say thy brand each
friend ' end and all thine show mine
eyes enclose view ' spent near beard grow
will both sheaves pride bright fear thee else
due due thee all eyes should brought desired
love decay kind and thee good fear sheds
shown stand thee thought thee thee ' grief
sort spent ' brought thee thee her sight
rage see cheeks control thee so great praise
plot out of truth repair thee ' it
be cheeks force constant fear torn view cheeks


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
